In [60]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import selenium.webdriver.support.ui as ui
import time
import os
import re

# wd = webdriver.Chrome(executable_path=r"C:\Users\Jedrek\Documents\wonga_interest_monitoring\chromedriver.exe")
EXEC_PATH = r"C:\Users\Jedrek\Documents\wonga_interest_monitoring\chromedriver.exe"
DOWNLOAD_PATH = "D:\\dokumenty\\KLAN\\"

In [61]:
def set_up_webdriver(exec_path, download_path):
    path = os.path.abspath(download_path)
    prefs = {"download.default_directory":path}
    options = Options()
    options.add_experimental_option("prefs", prefs)
    wd = webdriver.Chrome(executable_path=exec_path, options = options)
    return wd


In [62]:
def checkEqual(L1, L2):
    return len(L1) == len(L2) and sorted(L1) == sorted(L2)

In [63]:
def download_wait(directory, timeout, expected):
    """
    Wait for downloads to finish with a specified timeout.

    Args
    ----
    directory : str
        The path to the folder where the files will be downloaded.
    timeout : int
        How many seconds to wait until timing out.
    nfiles : int, defaults to None
        If provided, also wait for the expected number of files.

    """
    seconds = 0
    dl_wait = True
    while dl_wait and seconds < timeout:
        time.sleep(1)
        dl_wait = False
        files = os.listdir(directory)
        
        ep_number_found = []
        for fname in files:
            for number in expected:
                if number in fname:
                    ep_number_found.append(number)
        dl_wait = not checkEqual(expected, ep_number_found)
        seconds += 1
        set_E = set(expected)
        set_F = set(ep_number_found)
        if seconds % 15 == 0:
            print("Expected, but not found: ", set_E.difference(set_F))
            print("Found, but unexpected: ", set_F.difference(set_E))
    return seconds

In [64]:
def download_season(Season_number):
    start = time.time()
    ## THE PATH TO DOWNLOAD INTO
    download_path_s = DOWNLOAD_PATH + str(Season_number)
    
    ## LOOK INTO THE DOWNLOAD PATH, MAYBE SOME EPISODES ARE ALREADY THERE
    files = os.listdir(download_path_s)
    downloaded_episodes = [str(int(re.findall(r'E(\d{4}|\d{3})', i)[0])) for i in files]
    
    ## WEBDRIVER SETUP
    wd = set_up_webdriver(EXEC_PATH, download_path_s)
    
    ## request and transform the list of downloads
    page = f'http://staryklan.cba.pl/sezon{Season_number}.html'
    r = requests.get(page)
    txt = r.text
    soup = BeautifulSoup(txt, 'html.parser')
    li = soup.get_text().split('Odcinek')
    li2 = [i.strip().replace('\n', '') for i in li][1:]
    di = dict([i.split(":",1) for i in li2])
    
    ## initialize loop
    counter = 0
    logger = dict.fromkeys(di.keys())
    for key in di:
            if key[:3] not in downloaded_episodes:
                wd.switch_to.window(wd.window_handles[counter])
                url = di[key][:73] # hack, this is the proper length of download link
                try:
                    wd.get(url)
                    btn = ui.WebDriverWait(wd, 30).until(lambda wd: wd.find_element_by_xpath('//*[@id="startholder"]/div[2]/div[2]/section/div[1]/div/div[4]/div[3]/div[2]/div[1]/button'))
                    btn.click()
                    logger[key] = 'CLICKED'
                except BaseException as e:
                    logger[key] = e

                wd.execute_script("window.open(''),'_blannk'")
                counter += 1
                print(f'Odcinek {key}')
            else:
                print(f"Odcinek {key} has already been downloaded!")
            
            if counter % 3 == 0 and counter != 0:               
                
                nap = 200
                
                print(f'For a maximum of {nap} seconds I will be checking how the download is doing...')

                
                clicked_episodes = [k[:3] for k,v in logger.items() if v == 'CLICKED']
                clicked_or_downloaded = clicked_episodes + downloaded_episodes 
                
                elapsed = download_wait(download_path_s, nap, clicked_or_downloaded)
                
                if elapsed == nap:
                    print(f'The download is taking too long, something is wrong here...')
                else:
                    print(f'Slept for {elapsed}, back to work')
                    
                time.sleep(5)
                wd.quit()
                wd = set_up_webdriver(EXEC_PATH, download_path_s)
                counter = 0
                       
    end = time.time()
    print(f'Phew! That was a lot of work, it took {end - start}')
    
    return di, logger


In [ ]:
season_dict_4, logger_4 = download_season(4)

Odcinek 364 już jest ściągnięty!
Odcinek 365 już jest ściągnięty!
Odcinek 366 już jest ściągnięty!
Odcinek 367 już jest ściągnięty!
Odcinek 368-369 już jest ściągnięty!
Odcinek 370 już jest ściągnięty!
Odcinek 371 już jest ściągnięty!
Odcinek 372 już jest ściągnięty!
Odcinek 373 już jest ściągnięty!
Odcinek 374 już jest ściągnięty!
Odcinek 375 już jest ściągnięty!
Odcinek 376-377 już jest ściągnięty!
Odcinek 378 już jest ściągnięty!
Odcinek 379 już jest ściągnięty!
Odcinek 380 już jest ściągnięty!
Odcinek 381 już jest ściągnięty!
Odcinek 382 już jest ściągnięty!
Odcinek 383 już jest ściągnięty!
Odcinek 384 już jest ściągnięty!
Odcinek 385 już jest ściągnięty!
Odcinek 386 już jest ściągnięty!
Odcinek 387 już jest ściągnięty!
Odcinek 388 już jest ściągnięty!
Odcinek 389 już jest ściągnięty!
Odcinek 390 już jest ściągnięty!
Odcinek 391-392 już jest ściągnięty!
Odcinek 393 już jest ściągnięty!
Odcinek 394 już jest ściągnięty!
Odcinek 395
Odcinek 396
Odcinek 397
For a maximum of 200 seconds

In [59]:
    download_path_s = DOWNLOAD_PATH + str(4)
    
## LOOK INTO THE DOWNLOAD PATH, MAYBE SOME EPISODES ARE ALREADY THERE
    files = os.listdir(download_path_s)
# downloaded_episodes = [str(int(re.findall(r'E(\d{4})', i)[0])) for i in files]
# for i in files:
#     try:
#         print(str(int(re.findall(r'E(\d{4}|\d{3})', i)[0])))
#     except:
#         print(re.findall(r'E(\d{4} |\d{3}-\d{3})', i))
#         print(i)

    page = f'http://staryklan.cba.pl/sezon{4}.html'
    r = requests.get(page)
    txt = r.text
    soup = BeautifulSoup(txt, 'html.parser')
    li = soup.get_text().split('Odcinek')
    li2 = [i.strip().replace('\n', '') for i in li][1:]
    di = dict([i.split(":",1) for i in li2])
    [k[:3] for k,v in di.items() if v = 'https://mega.nz/file/dklE3DqA#LmEqhwXnAEraWurXHWfJTI5ENvWFZkZOBxxLnIjtLKk']

['364',
 '365',
 '366',
 '367',
 '370',
 '371',
 '372',
 '373',
 '374',
 '375',
 '376',
 '378',
 '379',
 '380',
 '381',
 '382',
 '383',
 '384',
 '385',
 '386',
 '387',
 '388',
 '389',
 '390',
 '391',
 '393',
 '394',
 '395',
 '396',
 '397',
 '398',
 '400',
 '401',
 '402',
 '403',
 '404',
 '405',
 '406',
 '407',
 '408',
 '409',
 '410',
 '411',
 '413',
 '414',
 '415',
 '417',
 '420',
 '421',
 '423',
 '426',
 '427',
 '428',
 '430',
 '431',
 '432',
 '433',
 '434',
 '435',
 '436',
 '437',
 '438',
 '439',
 '440',
 '441',
 '442',
 '443',
 '444',
 '445',
 '446',
 '447',
 '448',
 '449',
 '450',
 '451',
 '452',
 '453',
 '454',
 '455',
 '456',
 '457',
 '458',
 '459',
 '460',
 '461',
 '462',
 '463',
 '464',
 '465',
 '466',
 '467',
 '468',
 '469',
 '470',
 '471',
 '472',
 '473',
 '474',
 '475',
 '476',
 '477',
 '478',
 '479',
 '480',
 '481',
 '482',
 '483',
 '484',
 '485']

# BRUDNOPIS

In [10]:
arr = os.listdir(DOWNLOAD_PATH + '2')

import re

eps = [str(int(re.findall(r'E(\d{4})', i)[0])) for i in arr]
eps

IndexError: list index out of range

In [41]:
arr

['Klan.1998.E0112.PL.HDTV.x264-WujStefan (1).mkv',
 'Klan.1998.E0113.PL.HDTV.x264-WujStefan.mkv',
 'Klan.1998.E0114.PL.HDTV.x264-WujStefan (1).mkv',
 'Klan.1998.E0124.PL.HDTV.x264-WujStefan.mkv',
 'Klan.1998.E0125.PL.HDTV.x264-WujStefan.mkv',
 'Klan.1998.E0126.PL.HDTV.x264-WujStefan.mkv',
 'Klan.1998.E0127.PL.HDTV.x264-WujStefan.mkv',
 'Klan.1998.E0128.PL.HDTV.x264-WujStefan.mkv',
 'Klan.1998.E0134.PL.HDTV.x264-WujStefan.mkv']

In [47]:
logger = dict.fromkeys(di.keys())
logger['112']='CLICKED'
logger['113']='CLICKED'
clicked_episodes = [k for k,v in logger.items() if v == 'CLICKED']
cd = clicked_episodes + eps
ep_number_found = []
for file in arr:
    for number in cd:
        if number in file:
            ep_number_found.append(number)

In [54]:
print(cd)
print(ep_number_found)
checkEqual(cd,ep_number_found)

['112', '113', '112', '113', '114', '124', '125', '126', '127', '128', '134']
['112', '112', '113', '113', '114', '124', '125', '126', '127', '128', '134']


True

In [4]:
    page = f'http://staryklan.cba.pl/sezon{2}.html'
    r = requests.get(page)
    txt = r.text
    soup = BeautifulSoup(txt, 'html.parser')
    li = soup.get_text().split('Odcinek')
    li2 = [i.strip().replace('\n', '') for i in li][1:]
    di = dict([i.split(":",1) for i in li2])

In [7]:
for i in di:
    print(i[:3])

112
113
114
115
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238


In [19]:
for i in di.keys():
    if i in eps:
        print(i)

112
113
114
124
125
126
127
128
134


In [20]:
from collections import OrderedDict
from itertools import islice

di_O = OrderedDict(di)
ind = list(di_O.keys()).index('98')
sliced = islice(di_O.items(), ind, None)
OrderedDict(sliced)

OrderedDict([('98',
              'https://mega.nz/file/0NYHBCJI#h070Juoz8o824VR1KUDBGBLybO_4EJV9dFcbB6elASY'),
             ('99',
              'https://mega.nz/file/RJYx3QxB#dyUhhYogYlWsxCH8JzgFo7JxacJszzk8VkSFivjJad8'),
             ('100',
              'https://mega.nz/file/0YBDVKAb#zTESAnQCGHnCp5IzwbIxc2e9cEScJRkhiPsaGl4WAPI'),
             ('101-108',
              'https://mega.nz/file/oZAABTRb#3BppEATALePk9sm5_aMeD4exR3rTEjZ3TkViERV6Pq4'),
             ('109',
              'https://mega.nz/file/UNoUiLJS#Vu8q1wqIbg3NPWb5KRX1i6G6lBq9hWE3Z4FwRhL5D1o'),
             ('110',
              'https://mega.nz/file/8c5EiRQK#jdJ7zvDDmkcA-J4Qvxb-Orr-PBpcBxy5pN0nRCZA1ZI'),
             ('111',
              'https://mega.nz/file/VYp2wJJY#4KVJOhmkZ1v9hEKDIsIumgJLlqf-s_DTXH7rZn6xfDE')])

OrderedDict([('1',
              'https://mega.nz/file/NFBmBYya#0SgHd2-Yaqr331WhizBPpiFAIs876oC4q899AH9FEyo'),
             ('2',
              'https://mega.nz/file/ABQgHajB#78TTFqVrlIjFqkOK6D-w9omaG7EsxfxulUuhHYSIfMY'),
             ('3',
              'https://mega.nz/file/5VJ2QQRZ#sBtpJejaQ2MImjoBWbMujLbApEL6W8mvIh-PKFR5-yI'),
             ('4',
              'https://mega.nz/file/gRwRCIbQ#Q7_15awhDEwim606MMnGtZBg0a0y4HJS9YJhE74BOiQ'),
             ('5',
              'https://mega.nz/file/oJ4x2IBS#rZY6YBzI0eTPogcJGdKUk9j-7_4kNsWeXFGryEsEJ4I'),
             ('6',
              'https://mega.nz/file/hU4EjCoT#1v0K0h6GeK65nWS6GK8DaA_i_7bJ4kBlbwyaeypByvs'),
             ('7',
              'https://mega.nz/file/EA5QSATB#0iElHKFkrm1ovgSh6wlyC3vZ5Z7gtQOQ3xzvFdZUvlc'),
             ('8',
              'https://mega.nz/file/MY5UUayJ#55VAOb-9_5GC-1-1amEAq6LOPtdjYtDVxQhrfamhVlM'),
             ('9',
              'https://mega.nz/file/YEQTUSwC#o_fmfVIPBB8YqPl79mS7QdQ_WNXY0ik-guFNkDC20Hw'),
 

In [170]:
log = dict.fromkeys(di.keys())
log['1'] = 'DONE'
print(log)

{'1': 'DONE', '2': None, '3': None, '4': None, '5': None, '6': None, '7': None, '8': None, '9': None, '10': None, '11': None, '12': None, '13': None, '14': None, '15': None, '16': None, '17': None, '18': None, '19': None, '20': None, '21': None, '22': None, '23': None, '24': None, '25': None, '26': None, '27': None, '28': None, '29': None, '30': None, '31': None, '32': None, '33': None, '34': None, '35': None, '36': None, '37': None, '38': None, '39': None, '40': None, '41': None, '42': None, '43': None, '44': None, '45': None, '46': None, '47': None, '48': None, '49': None, '50': None, '51': None, '52': None, '53': None, '54': None, '55': None, '56': None, '57': None, '58': None, '59': None, '60': None, '61': None, '62': None, '63': None, '64': None, '65': None, '66': None, '67': None, '68': None, '69': None, '70': None, '71': None, '72': None, '73': None, '74': None, '75': None, '76': None, '77': None, '78': None, '79': None, '80': None, '81': None, '82': None, '83': None, '84': None

In [174]:
try:
    1 + '1'
except BaseException as e:
    print(e)

unsupported operand type(s) for +: 'int' and 'str'


In [72]:
sezony = [i for i in range(1,5)]
pages = []
for i in sezony:
    p = f'http://staryklan.cba.pl/sezon{i}.html'
    pages.append(p)

r = requests.get(pages[0])
txt = r.text

soup = BeautifulSoup(txt, 'html.parser')
li = soup.get_text().split('Odcinek')
li2 = [i.strip().replace('\n', '') for i in li][1:]
di = dict([i.split(":",1) for i in li2])


In [148]:
download_path_s = DOWNLOAD_PATH + str(1) 

wd = set_up_webdriver(EXEC_PATH, download_path_s)


page = 'https://mega.nz/file/NFBmBYya#0SgHd2-Yaqr331WhizBPpiFAIs876oC4q899AH9FEyo'
wd.get(page)
time.sleep(2)
btn = wd.find_element_by_xpath('//*[@id="startholder"]/div[2]/div[2]/section/div[1]/div/div[4]/div[3]/div[2]/div[1]/button')
btn.click()

In [146]:
download_path_s = DOWNLOAD_PATH + str(1) 
os.path.abspath(download_path_s)

'D:\\dokumenty\\KLAN\\1'

In [158]:
wd = set_up_webdriver(EXEC_PATH, download_path_s)

counter = 0

for i in [di['1'], di['2']]:
    wd.switch_to.window(wd.window_handles[counter])
    page = i
    wd.get(page)
#     time.sleep(2)
    btn = ui.WebDriverWait(wd, 15).until(lambda wd: wd.find_element_by_xpath('//*[@id="startholder"]/div[2]/div[2]/section/div[1]/div/div[4]/div[3]/div[2]/div[1]/button'))
    btn.click()
    wd.execute_script("window.open(''),'_blannk'")
    counter += 1
    
    

In [161]:
btn == first_result

True